In [1]:
# 1. 8-퍼즐을 너비 우선 탐색으로 해결
from math import sqrt 
from queue import Queue

start='806524731'
goal ='123456780'

def print_puzzle(p):
    n=int(sqrt(len(p)))
    print('\n'.join([p[n*i:n*(i+1)] for i in range(n)]),end='\n\n')

def Move(s,i,j):
    if '@' in s: return None 
    return s.replace(s[i],'@').replace(s[j],s[i]).replace('@',s[j])

def Branching(p):
    n=int(sqrt(len(p)))
    i=p.index('0')
    child=[]
    if not (i%n==0): child.append(Move(p,i,i-1))
    if not (i%n==n-1): child.append(Move(p,i,i+1))
    if i>=n: child.append(Move(p,i,i-n))
    if i<n*(n-1): child.append(Move(p,i,i+n))
    return child

print_puzzle(start)

Q=Queue()
Q.put(start)
V=[start]
while not Q.empty():
    node=Q.get()
    if node==goal:
        print(len(V),"개 노드를 방문하고 답을 찾았다.")
        break
    else: 
        child=Branching(node)
        for j in range(len(child)):
            if child[j] not in V:
                Q.put(child[j])
                V.append(child[j])



806
524
731

169973 개 노드를 방문하고 답을 찾았다.


In [5]:
# 2. 8-퍼즐을 너비 우선 탐색으로 해결(경로 출력 기능 추가)
from math import sqrt
from queue import Queue

start='806524731'
goal='123456780'

class Node():
    def __init__(self,state,path):
        self.state=state
        self.path=path

def print_puzzle(p):
    n=int(sqrt(len(p)))
    print('\n'.join([p[n*i:n*(i+1)] for i in range(n)]),end='\n\n')

def Move(s,i,j):
    if '@' in s: return None 
    return s.replace(s[i],'@').replace(s[j],s[i]).replace('@',s[j])

def Branching(node):
    p=node.state
    n=int(sqrt(len(p)))
    i=p.index('0')
    child=[]
    if not (i%n==0): child.append(Node(Move(p,i,i-1),node.path+'L'))
    if not (i%n==n-1): child.append(Node(Move(p,i,i+1),node.path+'R'))
    if i>=n: child.append(Node(Move(p,i,i-n),node.path+'U'))
    if i<n*(n-1): child.append(Node(Move(p,i,i+n),node.path+'D'))
    return child

print_puzzle(start)
Q=Queue()
root=Node(start,'-')
Q.put(root)
V=[root.state]
while not Q.empty():
    node=Q.get()
    if node.state==goal:
        print(len(V),'개 노드를 살피고 찾았다!')
        break
    else:
        child=Branching(node)
        for j in range(len(child)):
            if child[j].state not in V:
                Q.put(child[j])
                V.append(child[j].state)

print(f'{node.path} ({len(node.path)-1})')

806
524
731

169973 개 노드를 살피고 찾았다!
-DRULLDRDRULDLURULDDRUURDD (25)


In [21]:
# 번외 위에서 찾은 결과로 퍼즐 맞추기
from math import sqrt
from time import sleep
from IPython.display import clear_output
path = '-DRULLDRDRULDLURULDDRUURDD'
start = '806524731'

def Move_puzzle(s,i,direction):
    n=int(sqrt(len(s)))
    direction2idx ={'U':i-n,'D':i+n,'L':i-1,'R':i+1,'-':i}
    j = direction2idx[direction]
    if '@' in s: return None 
    return s.replace(s[i],'@').replace(s[j],s[i]).replace('@',s[j])

def print_puzzle(p):
    n=int(sqrt(len(p)))
    print('\n'.join([p[n*i:n*(i+1)] for i in range(n)]),end='\n\n')

def solve_puzzle(start,path,delay_time=3):
    puzzle = start
    for dir in path+'-':
        clear_output(wait=False)
        print(dir)
        print_puzzle(puzzle)
        puzzle = Move_puzzle(puzzle,puzzle.index('0'),dir)
        sleep(delay_time)

solve_puzzle(start,path)

-
123
456
780



In [9]:
# 3. 8-퍼즐을 최고 우선 탐색으로 해결(불일치 칸을 세는 평가 함수 사용)
from math import sqrt 
from queue import PriorityQueue

start='806524731'
goal='123456780'

n=int(sqrt(len(goal)))

class Node():
    def __init__(self,state,path):
        self.state=state
        self.path=path
        self.val=self.f()

    def f_mismatch(self):
        return sum(self.state[i]!=goal[i] for i in range(len(goal)))
    
    def f_manhattan(self):
        dist=0
        for i in range(len(goal)):
            r,c=i//n,i%n
            r1,c1=self.state.index(goal[i])//n,self.state.index(goal[i]%n)
            dist += abs(r1-r)+abs(c1-c)
        return dist
    
    def f(self): 
        return self.f_mismatch()
    
    def __lt__(self,other):
        return self.val<other.val

def print_puzzle(p):
    n=int(sqrt(len(p)))
    print('\n'.join([p[n*i:n*(i+1)] for i in range(n)]),end='\n\n')

def Move(s,i,j):
    if '@' in s: return None 
    return s.replace(s[i],'@').replace(s[j],s[i]).replace('@',s[j])

def Branching(node):
    p=node.state
    n=int(sqrt(len(p)))
    i=p.index('0')
    child=[]
    if not (i%n==0): child.append(Node(Move(p,i,i-1),node.path+'L'))
    if not (i%n==n-1): child.append(Node(Move(p,i,i+1),node.path+'R'))
    if i>=n: child.append(Node(Move(p,i,i-n),node.path+'U'))
    if i<n*(n-1): child.append(Node(Move(p,i,i+n),node.path+'D'))
    return child

print_puzzle(start)
Q=PriorityQueue()
root=Node(start,'-')
Q.put(root)
V=[root.state]
while not Q.empty():
    node=Q.get()
    if node.state==goal:
        print(len(V),'개 노드를 살피고 찾았다!')
        break
    else:
        child=Branching(node)
        for j in range(len(child)):
            if child[j].state not in V: 
                Q.put(child[j])
                V.append(node.state)
                
                
solve_puzzle(start,node.path)
print(f'{node.path} ({len(node.path)-1})')

806
524
731

1560 개 노드를 살피고 찾았다!
-DDRULDRUULDLURRDLDRUULDLDRUURDLDRUULDDLURURDLDRUULDDRUULDRULDLURRDLULDRRD (73)


In [22]:
# 3. 8-퍼즐을 최고 우선 탐색으로 해결(불일치 칸을 세는 평가 함수 사용)
from math import sqrt 
from queue import PriorityQueue

start='806524731'
goal='123456780'

n=int(sqrt(len(goal)))

class Node():
    def __init__(self,state,path):
        self.state=state
        self.path=path
        self.val=self.f()

    def f_mismatch(self):
        return sum(self.state[i]!=goal[i] for i in range(len(goal)))
    
    def f_manhattan(self):
        dist=0
        for i in range(len(goal)):
            r,c=i//n,i%n
            r1,c1=self.state.index(goal[i])//n,self.state.index(goal[i])%n
            dist += abs(r1-r)+abs(c1-c)
        return dist
    
    def f(self): 
        return self.f_manhattan()
    
    def __lt__(self,other):
        return self.val<other.val

def print_puzzle(p):
    n=int(sqrt(len(p)))
    print('\n'.join([p[n*i:n*(i+1)] for i in range(n)]),end='\n\n')

def Move(s,i,j):
    if '@' in s: return None 
    return s.replace(s[i],'@').replace(s[j],s[i]).replace('@',s[j])

def Branching(node):
    p=node.state
    n=int(sqrt(len(p)))
    i=p.index('0')
    child=[]
    if not (i%n==0): child.append(Node(Move(p,i,i-1),node.path+'L'))
    if not (i%n==n-1): child.append(Node(Move(p,i,i+1),node.path+'R'))
    if i>=n: child.append(Node(Move(p,i,i-n),node.path+'U'))
    if i<n*(n-1): child.append(Node(Move(p,i,i+n),node.path+'D'))
    return child

print_puzzle(start)
Q=PriorityQueue()
root=Node(start,'-')
Q.put(root)
V=[root.state]
while not Q.empty():
    node=Q.get()
    if node.state==goal:
        print(len(V),'개 노드를 살피고 찾았다!')
        break
    else:
        child=Branching(node)
        for j in range(len(child)):
            if child[j].state not in V: 
                Q.put(child[j])
                V.append(node.state)
                
                
solve_puzzle(start,node.path)
print(f'{node.path} ({len(node.path)-1})')

806
524
731

488 개 노드를 살피고 찾았다!
-DRDLURDLULURDDLURULDDRRUULDDLURRDLUURDDLLURRDLULDRR (51)


In [26]:
# 3. 8-퍼즐을 최고 우선 탐색으로 해결(불일치 칸을 세는 평가 함수 사용)
from math import sqrt 
from queue import PriorityQueue

start='806524731'
goal='123456780'

n=int(sqrt(len(goal)))

class Node():
    def __init__(self,state,path):
        self.state=state
        self.path=path
        self.val=self.f()

    def f_mismatch(self):
        return sum(self.state[i]!=goal[i] for i in range(len(goal)))
    
    def f_manhattan(self):
        dist=0
        for i in range(len(goal)):
            r,c=i//n,i%n
            r1,c1=self.state.index(goal[i])//n,self.state.index(goal[i])%n
            dist += abs(r1-r)+abs(c1-c)
        return dist
    
    def f_A_star(self):
        return len(self.path)+self.f_manhattan()
    
    def f(self): 
        return self.f_A_star()
    
    def __lt__(self,other):
        return self.val<other.val

def print_puzzle(p):
    n=int(sqrt(len(p)))
    print('\n'.join([p[n*i:n*(i+1)] for i in range(n)]),end='\n\n')

def Move(s,i,j):
    if '@' in s: return None 
    return s.replace(s[i],'@').replace(s[j],s[i]).replace('@',s[j])

def Branching(node):
    p=node.state
    n=int(sqrt(len(p)))
    i=p.index('0')
    child=[]
    if not (i%n==0): child.append(Node(Move(p,i,i-1),node.path+'L'))
    if not (i%n==n-1): child.append(Node(Move(p,i,i+1),node.path+'R'))
    if i>=n: child.append(Node(Move(p,i,i-n),node.path+'U'))
    if i<n*(n-1): child.append(Node(Move(p,i,i+n),node.path+'D'))
    return child

print_puzzle(start)
Q=PriorityQueue()
root=Node(start,'-')
Q.put(root)
V=[root.state]
while not Q.empty():
    node=Q.get()
    if node.state==goal:
        print(len(V),'개 노드를 살피고 찾았다!')
        break
    else:
        child=Branching(node)
        for j in range(len(child)):
            if child[j].state not in V: 
                Q.put(child[j])
                V.append(node.state)
                
                
solve_puzzle(start,node.path)
print(f'{node.path} ({len(node.path)-1})')

-
123
456
780

-DRULLDRDRULDLURULDDRUURDD (25)


In [11]:
# 6. 틱택토를 플레이하는 미니맥스 알고리즘
n=3
start='-'*(n*n)

def Move(state,pos,player):
    return state[:pos]+player+state[pos+1:]

def switch_player(player):
    return 'X' if player=='O' else 'O'

def print_board(state):
    print('  0123456789012345'[:n+2])
    for i in range(n):
        print(str(i%10)+':'+state[n*i:n*(i+1)])

def get_empty(state):
    if decide_winner(state) in ['O','X','T']:return []
    empty=[]
    for i in range(len(start)):
        if state[i]=='-':empty.append(i)
    return empty

def decide_winner(state):
    for (a,b,c) in [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]:
        if state[a]==state[b]==state[c]:
            if state[a]=='O': return 'O'
            elif state[a]=='X': return 'X'
    if [i for i in range(n*n) if state[i]=='-']==[]: return 'T'
    return 'N'

def minimax(state,player,depth):
    winner=decide_winner(state)
    if winner=='X': return 1, None
    elif winner=='O': return -1,None
    elif winner=='T': return 0,None
    
    e=get_empty(state)
    if depth%2==0:
        vmax,bestpos=-100,None
        for pos in e: 
            v,_=minimax(Move(state,pos,player),switch_player(player),depth+1)
            if v>vmax: vmax,bestpos=v,pos
        return vmax,bestpos
    else: 
        vmin,bestpos=100,None
        for pos in e:
            v,_=minimax(Move(state,pos,player),switch_player(player),depth+1)
            if v<vmin:vmin,bestpos=v,pos
        return vmin,bestpos
        
def tictactoe_play(first_mover):
    state=start
    player=first_mover
    print_board(state)
    while True:
        if player=='X':
            val,pos=minimax(state,player,0)
        elif player=='O':
            x,y=input('사람 차례입니다. (x와 y를 공백 구분하여 입력하세요.)').split(' ')
            pos=int(y)*n+int(x)
            if state[pos]!='-': 
                print('둘 수 없는 곳입니다.')
                continue
        state=Move(state,pos,player)
        print_board(state)
        winner=decide_winner(state)
        if winner in ['O','X','T']:
            if winner=='T':print('비겼습니다.')
            else: print(f'{winner}가 이겼습니다.')
            break
        player=switch_player(player)

tictactoe_play('O')

  012
0:---
1:---
2:---
  012
0:---
1:-O-
2:---
  012
0:X--
1:-O-
2:---
  012
0:X-O
1:-O-
2:---
  012
0:X-O
1:-O-
2:X--
  012
0:X-O
1:OO-
2:X--
  012
0:X-O
1:OOX
2:X--
  012
0:XOO
1:OOX
2:X--
  012
0:XOO
1:OOX
2:XX-
  012
0:XOO
1:OOX
2:XXO
비겼습니다.


In [3]:
# 7. 틱택토를 플레이하는 몬테카를로 트리 탐색 알고리즘
from math import sqrt,log
import random

n=3
start='-'*(n*n)

class Node():
    def __init__(self,state,player=None,pos=None,parent=None):
        self.state = state
        self.player = player
        self.pos = pos
        self.parent = parent
        self.nwin = 0
        self.nvisit = 0
        self.untried=get_empty(state)
        self.children=[]

    def UCTselect(self):
        s=sorted(self.children, key=lambda c: c.nwin/c.nvisit+sqrt(log(self.nvisit)/c.nvisit))
        return s[-1]

    def makeChild(self,state,pos,player):
        childnode=Node(state,player,pos,parent=self)
        self.untried.remove(pos)
        self.children.append(childnode)
        return childnode 
    
    def update(self,winner):
        self.nvisit+=1
        if winner=='T':self.nwin+=0.5
        elif winner==self.player:self.nwin+=1
    
    def __repr__(self):
        return str(self.state)+" "+str(self.nwin)+"/"+str(self.nvisit)

def Move(state,pos,player):
    return state[:pos]+player+state[pos+1:]

def switch_player(player):
    return 'X' if player=='O' else 'O'

def print_board(state):
    print('  01234567890123456'[:n+2])
    for i in range(n): print(str(i%10)+':'+state[n*i:n*(i+1)])
    
def get_empty(state):
    if decide_winner(state) in ['O','X','T']: return []
    empty=[]
    for i in range(len(start)):
        if state[i]=='-':empty.append(i)
    return empty

def decide_winner(state):
    for (a,b,c) in[(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]:
        if state[a]==state[b]==state[c]:
            if state[a]=='O': return 'O'
            elif state[b]=='X': return 'X'
    if [i for i in range(n*n) if state[i]=='-']==[]: return 'T'
    return 'N' 

def mcts(state,player): 
    root=Node(state)

    for i in range(10000):
        node = root
        state = node.state
        roll_player = player
        while node.untried==[] and node.children!=[]:
            node = node.UCTselect()
            state=Move(state,node.pos,roll_player)
            roll_player=switch_player(roll_player)
        
        if node.untried!=[]:
            pos=random.choice(node.untried)
            state = Move(state,pos,roll_player)
            node = node.makeChild(state,pos,roll_player)
            roll_player=switch_player(roll_player)
            
        while True:
            e=get_empty(state)
            if e==[]:break
            state=Move(state,random.choice(e),roll_player)
            roll_player=switch_player(roll_player) 
        
        winner=decide_winner(state)
        while node!=None:
            node.update(winner)
            node = node.parent
    return sorted(root.children,key=lambda c: c.nwin/c.nvisit)[-1].pos

def tictactoe_play(first_mover):
    state = start
    player=first_mover
    print_board(state)
    while True:
        if player=='X':
            print("컴퓨터 차례입니다.")
            pos = mcts(state,player)
        elif player=='O':
            x,y=input("사람 차례입니다.(x와 y를 공백 구분하여 입력하세요.)").split()
            pos = int(y)*n+int(x)
            if state[pos]!='-':
                print("둘 수 없는 곳입니다.")
                continue
        state = Move(state,pos,player)
        print_board(state)
        winner=decide_winner(state)
        if winner in ['O','X','T']:
            if winner=='T':print('비겼습니다.')
            else: print(winner,'가 이겼습니다.')
            break
        player=switch_player(player)

tictactoe_play('O')

  012
0:---
1:---
2:---
  012
0:---
1:-O-
2:---
컴퓨터 차례입니다.
  012
0:X--
1:-O-
2:---
  012
0:X--
1:-O-
2:--O
컴퓨터 차례입니다.
  012
0:X--
1:-O-
2:X-O
  012
0:X--
1:OO-
2:X-O
컴퓨터 차례입니다.
  012
0:X--
1:OOX
2:X-O
  012
0:XO-
1:OOX
2:X-O
컴퓨터 차례입니다.
  012
0:XO-
1:OOX
2:XXO
  012
0:XOO
1:OOX
2:XXO
비겼습니다.


In [4]:
# 8. 오목을 두는 몬테카를로 트리 탐색 알고리즘
from math import sqrt,log
import numpy as np
import random 
import time 
n=10
k=5
start='-'*(n*n)

class Node():
    def __init__(self,state,player=None,pos=None,parent=None):
        self.state=state
        self.player=player
        self.pos=pos
        self.parent=parent
        self.nwin=0
        self.nvisit=0
        self.untried=get_empty(state)
        self.children=[]
        
    def UCTselect(self):
        s=sorted(self.children,key=lambda c: c.nwin/c.nvisit+sqrt(log(self.nvisit)/c.nvisit))
        return s[-1]
    
    def makeChild(self,state,pos,player):
        node=Node(state,player,pos,parent=self)
        self.untried.remove(pos)
        self.children.append(node)
        return node
    
    def update(self,winner):
        self.nvisit+=1
        if winner=='T': self.nwin+=0.5
        elif winner==self.player: self.nwin+=1
    
    def __repe__(self):
        return f'{self.state} {self.nwin}/{self.nvisit}'

def randomAroundCenter(n):
    c=n*(n//2)+n//2
    return random.choice([c,c-1,c+1,c-n,c+n])

def Move(state,pos,player):
    return state[:pos]+player+state[pos:]

def switch_player(player):
    return 'X' if player=='O' else 'O'

def print_board(state):
    print('  0123456789012345'[:n+2])
    for i in range(n): print(f'{i%10}:{state[n*i:n*(i+1)]}')

def yx2pos(x,y):return y*n+x
def pos2yx(pos):return pos//n,pos%n

def get_empty(state):
    if decide_winner(state) in['O','X','T']: return []

    empty=[]
    for i in range(len(start)):
        if state[i]=='-': continue
        r,c=pos2yx(i)
        for (y,x) in [(r+i,c+j) for i in [-1,0,1] for j in [-1,0,1] if i!=0 or j!=0]:
            pos = y*n+x
            if 0<=x<n and 0<=y<n and state[pos]=='-' and pos not in empty: empty.append(pos)
    return empty
        
def decide_winner(state):
    nvoid = 0
    for i in range(n*n):
        s=state[i]
        if s=='-':
            nvoid+=1
            continue
        
        r,c=pos2yx(i)

        (r1,c1),(r2,c2) = [(r+i,c+i) for i in [-1,1]]

        while c1>=0 and state[yx2pos(r,c1)]==s:c1-=1
        while c2<n and state[yx2pos(r,c2)]==s:c2+=1
        if c2-c1-1==k: return s

        while r1>=0 and state[yx2pos(r1,c)]==s:r1-=1
        while r2<n and state[yx2pos(r2,c)]==s:r2+=1
        if r2-r1-1==k: return s 
        
        (r1,c1),(r2,c2) = [(r+i,c+i) for i in [-1,1]]
        while r1>=0 and c1>=0 and state[yx2pos(r1,c1)]==s:r1,c1=r1-1,c1-1
        while r2<n and c2<n and state[yx2pos(r2,c2)]==s: r2,c2=r2+1,c2+1
        if r2-r1-1==k: return s 

        (r1,c2),(r2,c1) = [(r+i,c+i) for i in [-1,1]]
        while r1>=0 and c1<n and state[yx2pos(r1,c1)]==s:r1,c1=r1-1,c1+1
        while r2<n and c2>=0 and state[yx2pos(r2,c2)]==s: r2,c2=r2+1,c2-1
        if r2-r1-1==k: return s 

    if nvoid==0: return 'T'
    return 'N'

def mcts(state,player):
    root = Node(state)
    
    for i in range(10000):
        node=root
        state=node.state
        roll_player=player
        while node.untried==[] and node.children!=[]:
            node=node.UCTselect()
            state=Move(state,node.pos,roll_player)
            roll_player=switch_player(roll_player)
        
        if node.untried!=[]:
            pos=random.choice(node.untried)
            state=Move(state,pos,roll_player)
            node=node.makeChild(state,pos,roll_player)
            roll_player=switch_player(roll_player)
            
        while True:
            e
        



        


    